In [2]:
!pip install boto3 -qq

In [1]:
# acessando o bucket do s3
import boto3
import os
import dotenv

dotenv.load_dotenv()

aws_access_key_id = os.getenv('AWS_ACCESS_KEY_ID'),
aws_secret_access_key = os.getenv('AWS_SECRET_ACCESS_KEY')
if not aws_access_key_id or not aws_secret_access_key:
    raise ValueError("AWS_ACCESS_KEY_ID and AWS_SECRET_ACCESS_KEY must be set as environment variables")

s3 = boto3.client('s3')
response = s3.list_buckets()

print("Buckets disponíveis:")
for bucket in response['Buckets']:
    print(f"- {bucket['Name']}")

s3_client = boto3.client('s3')
bucket_name = 'animaleproductsimages'

Buckets disponíveis:
- animaleproductsimages


In [2]:
def list_images_in_s3(s3_client, bucket_name, log=False):
    response = s3_client.list_objects_v2(Bucket=bucket_name)
    total_size = 0
    num_files = 0
    images = []
    if 'Contents' in response:
        for item in response['Contents']:
            images.append(item['Key'])
            total_size += item['Size']
            num_files += 1
        if log:
            print(f"> Total de arquivos: {num_files}, Tamanho total: {total_size/1024**2:.2f} MB")
    else:
        if log:
            print("O bucket está vazio.")
    return images

# Execute a listagem
file_list = list_images_in_s3(s3_client, bucket_name, log=True)
file_list


> Total de arquivos: 1, Tamanho total: 0.03 MB


['EssIconAgua.png']

In [42]:
def delete_image_from_bucket(s3_client, bucket_name, file_name):
    try:
        s3_client.delete_object(Bucket=bucket_name, Key=file_name)
        print(f'> {file_name} foi excluído do bucket com sucesso.')
    except Exception as e:
        print(f"Erro ao excluir {file_name}: {e}")

def delete_all_images_from_bucket(s3_client, bucket_name):
    try:
        response = s3_client.list_objects_v2(Bucket=bucket_name)
        if 'Contents' in response:
            for item in response['Contents']:
                s3_client.delete_object(Bucket=bucket_name, Key=item['Key'])
                print(f"{item['Key']} foi excluído com sucesso.")
        else:
            print("O bucket está vazio.")
    except Exception as e:
        print(f"Erro ao excluir imagens: {e}")

# Exemplo de uso
delete_image_from_bucket(s3_client, bucket_name, 'teste.png')

# Execute a exclusão de todas as imagens
delete_all_images_from_bucket(s3_client, bucket_name)

> teste.png foi excluído do bucket com sucesso.
blazer_blazer-alfaiataria-est-onca-jacquard_bege_03-04-0464-0030 foi excluído com sucesso.
blazer_blazer-alfaiataria-est-onca-jacquard_bege_03-04-0464-0030.jpg foi excluído com sucesso.


In [43]:
list_images_in_s3(s3_client, bucket_name, log=True)

O bucket está vazio.


[]

In [44]:
from pathlib import Path

# Função para upload de imagens
def upload_images_to_s3(s3_client, bucket_name, file_name, image_path):
    # for image_path in Path(local_folder).glob('*.png'):  # Altere para o formato desejado (jpg, png, etc.)
    #     file_name = image_path.name
        try:
            s3_client.upload_file(str(image_path), bucket_name, file_name)
            print(f'{file_name} foi enviado para o bucket com sucesso.')
        except Exception as e:
            print(f'Erro ao enviar {file_name}: {e}')

In [45]:
# Puxando arquivo parquet com os file_names
import pandas as pd

df_file_names = pd.read_parquet('datasets/base_animale_filtrada.parquet')

In [46]:
df_file_names

,file_path,id_produto,grupo_produto,file_name
0,animale_images_cropped/blazer/blazer_blazer-al...,03-04-0464-0030,blazer,blazer_blazer-alfaiataria-est-onca-jacquard_be...
1,animale_images_cropped/blazer/blazer_blazer-al...,03-04-0478-6048,blazer,blazer_blazer-alongado-marrom-nozes-sem-lapela...
2,animale_images_cropped/blazer/blazer_blazer-ci...,03-04-0460-0003,blazer,blazer_blazer-cinque-off-white_off-white_03-04...
3,animale_images_cropped/blazer/blazer_blazer-ci...,03-04-0461-0005,blazer,blazer_blazer-cinque-preto_preto_03-04-0461-00...
4,animale_images_cropped/blazer/blazer_blazer-co...,03-04-0479-0024,blazer,blazer_blazer-com-transpasse-de-crepe-off-whit...
...,...,...,...,...
761,animale_images_cropped/vestido/vestido_vestido...,07-20-6778-0005,vestido,vestido_vestido-tricot-transparencia_preto_07-...
762,animale_images_cropped/vestido/vestido_vestido...,07-20-6402-09248,vestido,vestido_vestido-tube_laranja-aura_07-20-6402-0...
763,animale_images_cropped/vestido/vestido_vestido...,07-20-6401-0005,vestido,vestido_vestido-tube_preto_07-20-6401-0005.jpg
764,animale_images_cropped/vestido/vestido_vestido...,07-20-6659-09538,vestido,vestido_vestido-tule-babados-lili-est-leopardo...


In [47]:
# Subindo base de imagens da animale
for file_name, file_path in zip(df_file_names['file_name'].to_list(), df_file_names['file_path'].to_list()):
    upload_images_to_s3(s3_client, bucket_name, file_name, file_path)

blazer_blazer-alfaiataria-est-onca-jacquard_bege_03-04-0464-0030.jpg foi enviado para o bucket com sucesso.
blazer_blazer-alongado-marrom-nozes-sem-lapela_marrom-nozes_03-04-0478-6048.jpg foi enviado para o bucket com sucesso.
blazer_blazer-cinque-off-white_off-white_03-04-0460-0003.jpg foi enviado para o bucket com sucesso.
blazer_blazer-cinque-preto_preto_03-04-0461-0005.jpg foi enviado para o bucket com sucesso.
blazer_blazer-com-transpasse-de-crepe-off-white_off-white_03-04-0479-0024.jpg foi enviado para o bucket com sucesso.
blazer_blazer-cropped-uno-amarelo-narciso_amarelo-narciso_03-04-0462-09256.jpg foi enviado para o bucket com sucesso.
blazer_blazer-de-algodao-khaki_caqui_03-04-0447-0278.jpg foi enviado para o bucket com sucesso.
blazer_blazer-de-couro-acinturado-botoes-organicos_bege-whitecap_03-01-1628-04346.jpg foi enviado para o bucket com sucesso.
blazer_blazer-de-crepe-bege-cropped_bege-manha_03-04-0431-09041.jpg foi enviado para o bucket com sucesso.
blazer_blazer-de-c

In [48]:
list_images_in_s3(s3_client, bucket_name, log=True)

> Total de arquivos: 766, Tamanho total: 9.66 MB


['blazer_blazer-alfaiataria-est-onca-jacquard_bege_03-04-0464-0030.jpg',
 'blazer_blazer-alongado-marrom-nozes-sem-lapela_marrom-nozes_03-04-0478-6048.jpg',
 'blazer_blazer-cinque-off-white_off-white_03-04-0460-0003.jpg',
 'blazer_blazer-cinque-preto_preto_03-04-0461-0005.jpg',
 'blazer_blazer-com-transpasse-de-crepe-off-white_off-white_03-04-0479-0024.jpg',
 'blazer_blazer-cropped-uno-amarelo-narciso_amarelo-narciso_03-04-0462-09256.jpg',
 'blazer_blazer-de-algodao-khaki_caqui_03-04-0447-0278.jpg',
 'blazer_blazer-de-couro-acinturado-botoes-organicos_bege-whitecap_03-01-1628-04346.jpg',
 'blazer_blazer-de-crepe-bege-cropped_bege-manha_03-04-0431-09041.jpg',
 'blazer_blazer-de-crepe-off-white_off-white_03-04-0491-0003.jpg',
 'blazer_blazer-de-crepe-vermelho-fiery_vermelho-fiery_03-04-0484-8259.jpg',
 'blazer_blazer-de-linho-oversized-off-white_off-white_03-04-0465-0024.jpg',
 'blazer_blazer-de-linho-preto_preto_03-04-0483-0005.jpg',
 'blazer_blazer-laranja-aura-detalhe-botoes_laranja-a

In [51]:
def get_link(file_name):
    return f"https://animaleproductsimages.s3.us-east-1.amazonaws.com/{file_name}"

get_link(df_file_names['file_name'][:9].to_list()[8])

'https://animaleproductsimages.s3.us-east-1.amazonaws.com/blazer_blazer-de-crepe-bege-cropped_bege-manha_03-04-0431-09041.jpg'